In [18]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import euclidean_distances
import os

In [19]:
# STEP 1: Load and Preprocess Images
class ImageDataset:
    def __init__(self, root_dir):
        """
        Initialize dataset with image transformations for AlexNet
        
        Args:
        root_dir (str): Path to the directory containing images
        """
        # Standard image transformations for AlexNet
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])
        ])
        
        # Load images
        self.dataset = ImageFolder(root=root_dir, transform=self.transform)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx]

# STEP 1: Feature Extraction with AlexNet
class AlexNetFeatureExtractor:
    def __init__(self):
        """
        Initialize pre-trained AlexNet and extract feature hooks
        """
        # Load pre-trained AlexNet
        self.model = models.alexnet(pretrained=True)
        self.model.eval()
        
        # Dictionary to store activations
        self.activations = {}
        
        # Register hooks for specified layers
        self.layers = {
            'conv1': self.model.features[0],
            'conv2': self.model.features[3],
            'conv3': self.model.features[6],
            'conv4': self.model.features[8],
            'conv5': self.model.features[10],
            'fc6': self.model.classifier[1],
            'fc7': self.model.classifier[4]
        }
        
        self.hooks = {}
        for name, layer in self.layers.items():
            self.hooks[name] = layer.register_forward_hook(
                self._get_activation(name)
            )
        
    def _get_activation(self, name):
        """
        Create hook to capture layer activations
        
        Args:
        name (str): Name of the layer
        
        Returns:
        hook function to store activations
        """
        def hook(module, input, output):
            self.activations[name] = output.detach().cpu().numpy()
        return hook
    
    def extract_features(self, images):
        """
        Extract features for given images
        
        Args:
        images (torch.Tensor): Batch of preprocessed images
        
        Returns:
        dict: Activation vectors for each specified layer
        """
        # Forward pass to trigger hooks
        with torch.no_grad():
            self.model(images)
        
        # Vectorize activations
        feature_vectors = {}
        for name, activation in self.activations.items():
            # Flatten and normalize activations
            feature_vectors[name] = activation.reshape(activation.shape[0], -1)
        
        return feature_vectors
    
    def __del__(self):
        """
        Remove hooks when done
        """
        for hook in self.hooks.values():
            hook.remove()

In [20]:
# STEP 2: Create Representational Dissimilarity Matrix (RDM)
def create_rdm(feature_vectors):
    """
    Create Representational Dissimilarity Matrix
    
    Args:
    feature_vectors (np.ndarray): Feature vectors for images
    
    Returns:
    np.ndarray: Representational Dissimilarity Matrix
    """
    return euclidean_distances(feature_vectors)

In [21]:
# STEP 3: Visualization Functions
def plot_rdm(rdm, title, class_labels=None):
    """
    Plot Representational Dissimilarity Matrix
    
    Args:
    rdm (np.ndarray): Representational Dissimilarity Matrix
    title (str): Plot title
    class_labels (list, optional): Labels for each image class
    """
    plt.figure(figsize=(12, 10))
    sns.heatmap(rdm, cmap='viridis', square=True, cbar=True)
    plt.title(f'Representational Dissimilarity Matrix - {title}')
    plt.tight_layout()
    plt.savefig(f'rdm_{title}.png')
    plt.close()

def perform_mds(rdm, class_labels):
    """
    Perform Multidimensional Scaling
    
    Args:
    rdm (np.ndarray): Representational Dissimilarity Matrix
    class_labels (list): Labels for each image class
    
    Returns:
    np.ndarray: MDS coordinates
    """
    # Perform MDS
    mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    mds_coords = mds.fit_transform(rdm)
    
    # Create color map
    unique_labels = list(set(class_labels))
    color_map = plt.cm.get_cmap('viridis')
    
    plt.figure(figsize=(12, 10))
    for label in unique_labels:
        mask = np.array(class_labels) == label
        plt.scatter(
            mds_coords[mask, 0], 
            mds_coords[mask, 1], 
            label=f'Class {label}', 
            color=color_map(label/len(unique_labels))
        )
    
    plt.title('Multidimensional Scaling (MDS) Visualization')
    plt.xlabel('MDS Dimension 1')
    plt.ylabel('MDS Dimension 2')
    plt.legend()
    plt.tight_layout()
    plt.savefig('mds_visualization.png')
    plt.close()
    
    return mds_coords

In [22]:
# Main Execution Function
def main():
    # Image directory (replace with your actual path)
    image_dir = './Image Set'
    
    # Initialize dataset and feature extractor
    dataset = ImageDataset(image_dir)
    extractor = AlexNetFeatureExtractor()
    
    # Create data loader
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=len(dataset), shuffle=False
    )
    
    # STEP 1: Prepare Class Labels
    def assign_class_label(idx):
        if 0 <= idx < 28:
            return 0  # Animals
        elif 28 <= idx < 64:
            return 1  # Objects
        elif 64 <= idx < 100:
            return 2  # Scenes
        elif 100 <= idx < 124:
            return 3  # Human Activities
        else:
            return 4  # Faces
    
    class_labels = [assign_class_label(i) for i in range(len(dataset))]
    
    # STEP 1 & 2: Extract Features and Create RDM for each layer
    layer_rdms = {}
    for layer_name in extractor.layers.keys():
        # Get images and extract features
        images, _ = next(iter(dataloader))
        features = extractor.extract_features(images)[layer_name]
        
        # STEP 2: Create RDM
        rdm = create_rdm(features)
        layer_rdms[layer_name] = rdm
        
        # STEP 3: Visualizations
        # Plot RDM
        plot_rdm(rdm, layer_name, class_labels)
        
        # Perform MDS
        perform_mds(rdm, class_labels)

if __name__ == '__main__':
    main()

c:\Users\Haren\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Haren\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Haren\AppData\Local\Temp\ipykernel_15544\4288057105.py:35: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color_map = plt.cm.ge